<a href="https://colab.research.google.com/github/chaumintruong/AIO_Module2_Image_Depth_Estimation/blob/feature%2Fproject_IDE/Module02_Image_Depth_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip /content/tsukuba.zip

Archive:  /content/tsukuba.zip
   creating: tsukuba/tsukuba/
  inflating: tsukuba/tsukuba/groundtruth.png  
  inflating: tsukuba/tsukuba/left.png  
  inflating: tsukuba/tsukuba/right.png  


In [3]:
!unzip /content/aloe.zip

Archive:  /content/aloe.zip
   creating: aloe/aloe/
   creating: aloe/aloe/Aloe/
  inflating: aloe/aloe/Aloe/Aloe_left_1.png  
  inflating: aloe/aloe/Aloe/Aloe_right_1.png  
  inflating: aloe/aloe/Aloe/Aloe_right_2.png  
  inflating: aloe/aloe/Aloe/Aloe_right_3.png  
  inflating: aloe/aloe/Aloe/Disparity_map_corr.png  


#Problem 01:

In [4]:
import cv2
import numpy as np

In [5]:
def distance_l1(p1, p2):
    return abs(p1 - p2)

def distance_l2(p1, p2):
    return (p1 - p2) ** 2

In [6]:
def pixel_wise_matching_l1(left_img, right_img, disparity_range=16, save_result=True):
    # Đọc ảnh trái và phải, sau đó chuyển đổi sang grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo disparity map trống
    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                cost = max_value if (x - j) < 0 else distance_l1(int(left[y, x]), int(right[y, x - j]))

                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('pixel_wise_l1.png', depth)
        cv2.imwrite('pixel_wise_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth

def pixel_wise_matching_l2(left_img, right_img, disparity_range=16, save_result=True):
    # Đọc ảnh trái và phải, sau đó chuyển đổi sang grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo disparity map trống
    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                cost = max_value if (x - j) < 0 else distance_l2(int(left[y, x]), int(right[y, x - j]))

                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('pixel_wise_l2.png', depth)
        cv2.imwrite('pixel_wise_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth


In [8]:
left_img_path = 'tsukuba/left.png'
right_img_path = 'tsukuba/right.png'
disparity_range = 16

pixel_vise_result_l1 = pixel_wise_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    save_result=True)

pixel_vise_result_l2 = pixel_wise_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    save_result=True)

Saving result...
Done.
Saving result...
Done.


#Problem 2:


In [9]:
def window_based_matching_l1(left_img, right_img, disparity_range=64, kernel_size=3, save_result=True):
    # Đọc ảnh trái và phải, sau đó chuyển đổi sang grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo disparity map trống
    depth = np.zeros((height, width), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = 65534

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = distance_l1(
                                int(left[y + v, x + u]), int(right[y + v, (x + u) - j])
                            )
                        total += value

                if total < cost_min:
                    cost_min = total
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l1.png', depth)
        cv2.imwrite('window_based_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth

def window_based_matching_l2(left_img, right_img, disparity_range=64, kernel_size=3, save_result=True):
    # Đọc ảnh trái và phải, sau đó chuyển đổi sang grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo disparity map trống
    depth = np.zeros((height, width), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = 65534

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = distance_l2(
                                int(left[y + v, x + u]), int(right[y + v, (x + u) - j])
                            )
                        total += value

                if total < cost_min:
                    cost_min = total
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l2.png', depth)
        cv2.imwrite('window_based_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth


In [10]:
left_img_path = 'Aloe/Aloe_left_1.png'
right_img_path = 'Aloe/Aloe_right_1.png'
disparity_range = 64
kernel_size = 3

window_based_result_l1 = window_based_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True)

window_based_result_l2 = window_based_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True)

Saving result...
Done.
Saving result...
Done.


#Problem 3:

In [12]:
# Đường dẫn đến ảnh trái và phải
left_img = 'Aloe/Aloe_left_1.png'
right_img = 'Aloe/Aloe_right_2.png'
disparity_range = 64
kernel_size = 5

# Tính disparity map với cost L1
disparity_map_l1 = window_based_matching_l1(
    left_img,
    right_img,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True)

# Tính disparity map với cost L2
disparity_map_l2 = window_based_matching_l2(
    left_img,
    right_img,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True)


Saving result...
Done.
Saving result...
Done.


#Problem 4:

In [13]:
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator

In [14]:
def window_based_matching_cosine_similarity(left_img, right_img, disparity_range=64, kernel_size=5, save_result=True):
    # Đọc ảnh trái và phải, sau đó chuyển đổi sang grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo disparity map trống
    depth = np.zeros((height, width), np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_optimal = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1

                if (d - kernel_half) >= 0:
                    wp = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    wqd = right[(y - kernel_half):(y + kernel_half) + 1, (d - kernel_half):(d + kernel_half) + 1]

                    # Chuyển đổi các cửa sổ thành vector (flatten)
                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    # Tính toán Cosine Similarity
                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                # Lưu giá trị disparity tại điểm có Cosine Similarity cao nhất
                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j

            # Gán disparity cho ảnh depth
            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_cosine_similarity.png', depth)
        cv2.imwrite('window_based_cosine_similarity_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done.')
    return depth


In [16]:
# Đường dẫn đến ảnh trái và phải
left_img = 'Aloe/Aloe_left_1.png'
right_img = 'Aloe/Aloe_right_2.png'
disparity_range = 64
kernel_size = 5

# Tính disparity map sử dụng Cosine Similarity
disparity_map_cosine = window_based_matching_cosine_similarity(
    left_img,
    right_img,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True)


Saving result...
Done.
